# Hypothesis Testing

**Hypothesis**: University towns have their mean housing prices less effected by recessions.

In [25]:
import pandas as pd
import numpy as np
import re
from scipy.stats import ttest_ind

#### Returns a DataFrame of towns and the states

In [26]:
def get_list_of_university_towns():    
    universityTownsFile = open('university_towns.txt')
    universityTowns = universityTownsFile.readlines()
    universityTownsFile.close()
    df = pd.DataFrame(columns=["State", "RegionName"])
    state = ''
    i = 0
    for line in universityTowns:
        line = line.strip()
        if line[-6:] == "[edit]":
            state = line[:-6]
        else:
            df.loc[i] = [state, re.sub(r' \(.*', "", line)]
            i += 1
    
    return df

In [27]:
get_list_of_university_towns().head()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo


#### Returns the year and quarter of the recession start time as a string value

In [38]:
def get_recession_start(): 
    gdp = pd.read_excel(
        'gdplev.xls',
        header=None,
        skiprows=220, 
        usecols=[4,6], 
        names=['quarter', 'gdp']
    )
    
    for i in range(0, len(gdp)-1):
        if gdp.loc[i, 'gdp'] > gdp.loc[i+1, 'gdp'] > gdp.loc[i+2, 'gdp']:
            return gdp.loc[i, 'quarter']

In [39]:
get_recession_start()

'2008q2'

#### Returns the year and quarter of the recession end time as a string value 

In [40]:
def get_recession_end():       
    gdp = pd.read_excel(
        'gdplev.xls',
        header=None,
        skiprows=220,
        usecols=[4,6],
        names=['quarter', 'gdp']
    )
    gdp = gdp[gdp['quarter'] >= get_recession_start()]
    gdp = gdp.reset_index()
    for i in range(0, len(gdp)-1):
        if gdp.loc[i, 'gdp'] < gdp.loc[i+1, 'gdp'] < gdp.loc[i+2, 'gdp']:
            return gdp.loc[i+2, 'quarter']

In [41]:
get_recession_end()

'2009q4'

#### Returns the year and quarter of the recession bottom time as a string value

In [42]:
def get_recession_bottom():    
    gdp = pd.read_excel(
        'gdplev.xls',
        header=None,
        skiprows=220,
        usecols=[4,6], 
        names=['quarter', 'gdp']
    )
    gdp = gdp[gdp['quarter'] >= get_recession_start()]
    gdp = gdp.reset_index()
    for i in range(0, len(gdp)-1):
        if gdp.loc[i, 'gdp'] < gdp.loc[i+1, 'gdp'] < gdp.loc[i+2, 'gdp']:
            return gdp.loc[i, 'quarter']

In [43]:
get_recession_bottom()

'2009q2'

#### Converts the housing data to quarters and returns it as mean values

In [44]:
states = {'OH': 'Ohio', 
          'KY': 'Kentucky', 
          'AS': 'American Samoa', 
          'NV': 'Nevada', 
          'WY': 'Wyoming', 
          'NA': 'National', 
          'AL': 'Alabama', 
          'MD': 'Maryland', 
          'AK': 'Alaska', 
          'UT': 'Utah', 
          'OR': 'Oregon', 
          'MT': 'Montana', 
          'IL': 'Illinois', 
          'TN': 'Tennessee', 
          'DC': 'District of Columbia', 
          'VT': 'Vermont',
          'ID': 'Idaho',
          'AR': 'Arkansas',
          'ME': 'Maine',
          'WA': 'Washington',
          'HI': 'Hawaii',
          'WI': 'Wisconsin',
          'MI': 'Michigan',
          'IN': 'Indiana',
          'NJ': 'New Jersey',
          'AZ': 'Arizona',
          'GU': 'Guam',
          'MS': 'Mississippi',
          'PR': 'Puerto Rico',
          'NC': 'North Carolina',
          'TX': 'Texas',
          'SD': 'South Dakota',
          'MP': 'Northern Mariana Islands',
          'IA': 'Iowa',
          'MO': 'Missouri',
          'CT': 'Connecticut',
          'WV': 'West Virginia',
          'SC': 'South Carolina',
          'LA': 'Louisiana',
          'KS': 'Kansas',
          'NY': 'New York',
          'NE': 'Nebraska',
          'OK': 'Oklahoma',
          'FL': 'Florida',
          'CA': 'California',
          'CO': 'Colorado',
          'PA': 'Pennsylvania',
          'DE': 'Delaware',
          'NM': 'New Mexico',
          'RI': 'Rhode Island',
          'MN': 'Minnesota',
          'VI': 'Virgin Islands', 
          'NH': 'New Hampshire',
          'MA': 'Massachusetts',
          'GA': 'Georgia',
          'ND': 'North Dakota',
          'VA': 'Virginia'}

In [45]:
def convert_housing_data_to_quarters():    
    all_homes = pd.read_csv(
        'City_Zhvi_AllHomes.csv', 
        usecols=[1,2]+list(range(51,251))
    )
    all_homes['State'] = all_homes['State'].map(states)
    all_homes.set_index(["State","RegionName"], inplace=True)
    
    def quarters(col):
        if col.endswith(("01", "02", "03")):
            return col[:4] + "q1"
        elif col.endswith(("04", "05", "06")):
            return col[:4] + "q2"
        elif col.endswith(("07", "08", "09")):
            return col[:4] + "q3"
        elif col.endswith(("10", "11", "12")):
            return col[:4] + "q4"
    
    all_homes = all_homes.groupby(quarters, axis = 1).mean()
    
    return all_homes.sort_index()

In [46]:
convert_housing_data_to_quarters().head()

2000q1         2000q2         2000q3  \
State   RegionName                                                 
Alabama Adamsville    69033.333333   69166.666667   69800.000000   
        Alabaster    122133.333333  123066.666667  123166.666667   
        Albertville   73966.666667   72600.000000   72833.333333   
        Arab          83766.666667   81566.666667   81333.333333   
        Ardmore                NaN            NaN            NaN   

                            2000q4         2001q1         2001q2  \
State   RegionName                                                 
Alabama Adamsville    71966.666667   73466.666667   74000.000000   
        Alabaster    123700.000000  123233.333333  125133.333333   
        Albertville   74200.000000   75900.000000   76000.000000   
        Arab          82966.666667   84200.000000   84533.333333   
        Ardmore                NaN            NaN            NaN   

                            2001q3         2001q4         2002q1  \
State   RegionName                                                 
Alabama Adamsville    73333.333333   73100.000000   73333.333333   
        Alabaster    127766.666667  127200.000000  127300.000000   
        Albertville   72066.666667   73566.666667   76533.333333   
        Arab          81666.666667   83900.000000   87266.666667   
        Ardmore                NaN            NaN            NaN   

                            2002q2    ...            2014q2         2014q3  \
State   RegionName                    ...                                    
Alabama Adamsville    73133.333333    ...      77066.666667   75966.666667   
        Alabaster    128000.000000    ...     147133.333333  147633.333333   
        Albertville   76366.666667    ...      84033.333333   84766.666667   
        Arab          87700.000000    ...     113366.666667  111700.000000   
        Ardmore                NaN    ...     140533.333333  139566.666667   

                       2014q4         2015q1         2015q2         2015q3  \
State   RegionName                                                           
Alabama Adamsville    71900.0   71666.666667   73033.333333   73933.333333   
        Alabaster    148700.0  148900.000000  149566.666667  150366.666667   
        Albertville   86800.0   88466.666667   89500.000000   90233.333333   
        Arab         111600.0  110166.666667  109433.333333  110900.000000   
        Ardmore      140900.0  143233.333333  143000.000000  144600.000000   

                            2015q4         2016q1         2016q2    2016q3  
State   RegionName                                                          
Alabama Adamsville    73866.666667   74166.666667   74933.333333   74700.0  
        Alabaster    151733.333333  153466.666667  155100.000000  155850.0  
        Albertville   91366.666667   92000.000000   92466.666667   92200.0  
        Arab         112233.333333  110033.333333  110100.000000  112000.0  
        Ardmore      143966.666667  142566.666667  143233.333333  141950.0  

[5 rows x 67 columns]

#### Ttest comparing the university town values to the non-university towns values

In [47]:
def run_ttest():    
    recession_start = get_recession_start()
    recession_bottom = get_recession_bottom()
    all_housing = convert_housing_data_to_quarters()
    all_housing = all_housing[[recession_start, recession_bottom]]
    all_housing.dropna(inplace=True)
    all_housing['price_ratio'] = all_housing[recession_start]/all_housing[recession_bottom]
    uni_towns = get_list_of_university_towns()
    uni_towns.set_index(["State", "RegionName"], inplace=True)
    
    uni_housing = pd.merge(
        all_housing, 
        uni_towns, 
        how='inner', 
        left_index=True, 
        right_index=True
    )
    non_uni_housing = all_housing[~all_housing.index.isin(uni_towns.index)]
    
    t, p = ttest_ind(uni_housing['price_ratio'], non_uni_housing['price_ratio'])
    
    different = False
    
    if p < 0.01:
        different = True
        
    if t < 0:
        better = "university town"
    else:
        better = "non-university town"
    
    return (different, p, better)

In [48]:
run_ttest()

(True, 0.002724063704761164, 'university town')